In [1]:
import pycochleagram.cochleagram as cgram
from pycochleagram import utils

import json
import sys
import os 
from os import listdir

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNetCV, LinearRegression, RidgeCV
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms

from helper_functions import *


In [2]:
path="/Users/ariellerosinski/My Drive/UCL/MSc/Project/Code/full_coch"

sound_files=listdir(path)

file_name=[]
files=[]
for file in sound_files:
    if file.endswith(".npy") and file.startswith("full"):
        file_name.append(file)
        full_path=[path,file]
        full_path="/".join(full_path)
        files.append(full_path)


k_folds=5
kfold = KFold(n_splits=k_folds, shuffle=False) 
foldid=0
split_size = 30

train_data=get_windows(files, kfold, foldid, file_name, split_size=split_size, norm=True, training=True)[0]


all_data_temp=np.stack(train_data)
all_data = np.reshape(all_data_temp, (all_data_temp.shape[0], -1)) 

feat_cols = ['bin'+str(i) for i in range(all_data.shape[1])]
df_coch = pd.DataFrame(all_data,columns=feat_cols)

pca_coch = PCA() 
pca_coch.fit(df_coch.iloc[:,:])

PC_cols = ['PC'+str(i) for i in range(pca_coch.components_.shape[1])]
principal_coch_Df = pd.DataFrame(pca_coch.components_, columns = PC_cols)

PCs = principal_coch_Df.to_numpy().reshape(-1,81,split_size)
np.save("/path to saved PCs ", PCs)